In [1]:
import logging
logging.getLogger().setLevel(logging.INFO)
logging.captureWarnings(True)

import json
import pkg_resources

import numpy as np

import descarteslabs as dl
from descarteslabs import workflows as wf
from tqdm.notebook import tqdm
import pandas as pd
from shapely import geometry as sgeom
from IPython import display as dsp
import shapely.geometry as sg
import will_utils as utils
import geojson

In [2]:
# Get shape of Yolo county in California
places_client = dl.Places()
state = places_client.shape("north-america_united-states_california_sacramento-valley_yolo")

# Define Workflows GeoContext
geoctx = dl.scenes.AOI(sg.shape(state.geometry).simplify(0.01),
                       crs="EPSG:4326", resolution=0.01)

# Display on a map:
dsp.GeoJSON(geojson.Feature(geometry=geoctx.geometry))

<IPython.display.GeoJSON object>

Here we start to define the workflow. We will use Sentinel-2 imagery which has a resolution of 10 meters. In this next block of code, comments have been added before each step explaining what is happening in more detail.

### Weather

In this section we are calculating the Growing Degree Days, Precipitation, and Soil Moisture content in a daily intervals for the entire year. We define a basic workflow object below and then expand on it for each of the layers we are going to compute.

In [3]:
start_datetime='2018-01-01'
end_datetime='2021-10-01'

ncepflow = wf.ImageCollection.from_id('ncep:cfsr-v2:daily:v1', 
                                       start_datetime=start_datetime, 
                                       end_datetime=end_datetime)

#### Growing Degree Days

In [4]:
# https://en.wikipedia.org/wiki/Growing_degree-day#GDD_calculation
# NCEP data is in 0.01 K
tmin, tmax =  ncepflow.unpack_bands('tmin tmax')

#convert 50 degrees to kelvin and then add it to 283
gdd_base = 283.
gdd = (((tmax + tmin) / (2*100)) - gdd_base).clip_values(min=0.)
gdd_ts = gdd.map(lambda img: wf.Dict[wf.Datetime, wf.Float].from_pairs([(img.properties['date'], img.median('pixels')['tmax_add_tmin'])]))

In [5]:
gdd_res = gdd_ts.compute(geoctx)


Job ID: 75ea457c28bddaf41c50bcc1456e9cf43f6d017bd92417ea
[######] | Steps: 8202/8202 | Stage: SUCCEEDED                                

In [6]:
gdd_dates = []
gdd_vals = []
for res in gdd_res:
    key = list(res.keys())[0]
    gdd_dates.append(pd.to_datetime(key.split('T')[0], format='%Y-%m-%d'))
    gdd_vals.append(res[key])

In [7]:
# Save these lists to text files for further processing
with open('texts/pres_gdd_dates.txt', 'w') as f:
    for item in gdd_dates:
        f.write("%s\n" % item)

with open('texts/pres_gdd_vals.txt', 'w') as f:
    for item in gdd_vals:
        f.write("%s\n" % item)

#### Precipitation

In [8]:
precip = ncepflow.pick_bands('prec')
precip_ts = precip.map(lambda img: wf.Dict[wf.Datetime, wf.Float].from_pairs([(img.properties['date'], img.max('pixels')['prec'])]))

In [9]:
precip_res = precip_ts.compute(geoctx)


Job ID: bf93ec721b9a1e0d5059fa113e5b05dc3bbd034a10c98f25
[######] | Steps: 5468/5468 | Stage: SUCCEEDED                                

In [10]:
precip_dates = []
precip_vals = []
for res in precip_res:
    key = list(res.keys())[0]
    precip_dates.append(pd.to_datetime(key.split('T')[0], format='%Y-%m-%d'))
    precip_vals.append(res[key] * 0.1)

In [11]:
# Save these lists to text files for further processing
with open('texts/pres_precip_dates.txt', 'w') as f:
    for item in precip_dates:
        f.write("%s\n" % item)

with open('texts/pres_precip_vals.txt', 'w') as f:
    for item in precip_vals:
        f.write("%s\n" % item)

#### Soil Moisture

In [12]:
soil_moisture = ncepflow.pick_bands('soilmoist2')
soil_moisture_ts = soil_moisture.map(lambda img: wf.Dict[wf.Datetime, wf.Float].from_pairs([(img.properties['date'], img.median('pixels')['soilmoist2'])]))

In [13]:
soil_res = soil_moisture_ts.compute(geoctx)


Job ID: ee3ee6f8c1fd5a3c0a021d1ee5539d8480a19ab4437e6293
[######] | Steps: 5468/5468 | Stage: SUCCEEDED                                

In [14]:
soil_dates = []
soil_vals = []
for res in soil_res:
    key = list(res.keys())[0]
    soil_dates.append(pd.to_datetime(key.split('T')[0], format='%Y-%m-%d'))
    soil_vals.append(res[key] * 0.01)

In [15]:
# Save these lists to text files for further processing
with open('texts/pres_soil_dates.txt', 'w') as f:
    for item in soil_dates:
        f.write("%s\n" % item)

with open('texts/pres_soil_vals.txt', 'w') as f:
    for item in soil_vals:
        f.write("%s\n" % item)

Now lets plot the results

In [ ]:
ylabs = ['NDVI', 'GDDs', 'Precipitation (mm)', 'Soil Moisture (%)']

fig, axs = plt.subplots(4, 1, figsize=(14, 10), sharex=True)
#axs[0].plot(group_dates, ndvi_vals)
axs[1].plot(gdd_dates, gdd_vals)
axs[2].plot(precip_dates, precip_vals)
axs[3].plot(soil_dates, soil_vals)

for (ax, lab) in zip(axs, ylabs):
    ax.set_ylabel(lab, size=13)
    ax.tick_params('both', labelsize=12)

axs[-1].set_xlabel('Date', size=13)